<a href="https://colab.research.google.com/github/AdityaArakere/Sentiment-analysis-model/blob/main/FirstTenserflowModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import string
import re
from google.colab import files


uploaded = files.upload()


Saving combined_dataset7.csv to combined_dataset7.csv


In [ ]:
import pandas as pd
import io

df = pd.read_csv(io.BytesIO(uploaded['combined_dataset7.csv']))
df = df[['text', 'sentiment']]

# Preprocess the dataset
# Convert sentiment labels to 0, 1, -1
# df['category'] = df['category'].map({-1: [1, 2], 0: [3, 0], 1: [4, 5]})

# Convert text to lowercase
df['text'] = df['text'].str.lower()

# Remove punctuation
translator = str.maketrans('', '', string.punctuation)
df['text'] = df['text'].astype(str).apply(lambda x: x.translate(translator))

# Remove numbers
df['text'] = df['text'].apply(lambda x: re.sub(r'\d+', '', x))

# Remove extra whitespaces
df['text'] = df['text'].apply(lambda x: ' '.join(x.split()))

# Remove stopwords (optional)
# You can use NLTK or other libraries to remove stopwords if desired

# Print the preprocessed dataset
print(df.head())


                                                text  sentiment
0  one of the other reviewers has mentioned that ...          1
1  a wonderful little production br br the filmin...          1
2  i thought this was a wonderful way to spend ti...          1
3  petter matteis love in the time of money is a ...          1
4  probably my alltime favorite movie a story of ...          1


In [ ]:
from sklearn.model_selection import train_test_split
import tensorflow as tf
!pip install transformers
from transformers import BertTokenizer, TFBertForSequenceClassification
from tensorflow.keras.layers import Input, Dense, Dropout, GlobalMaxPool1D

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 46.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 77.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 66.6 MB/s eta 0:00:00


In [ ]:
# Split the dataset into train and test sets
x = df['text'].values
y = df['sentiment'].values

x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=4000, test_size=800, random_state=42)

# Print the sizes of the train and test sets
print("Train set size:", len(x_train))
print("Test set size:", len(x_test))
print(y_train[0:5])

Train set size: 4000
Test set size: 800
[1 0 0 0 1]


In [ ]:
# Define a custom dataset class
class RedditDataset(tf.keras.utils.Sequence):
    def __init__(self, texts, labels, tokenizer, batch_size):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.batch_size = batch_size

        self.on_epoch_end()

    def __len__(self):
        return len(self.texts) // self.batch_size

    def __getitem__(self, index):
        batch_texts = self.texts[index*self.batch_size:(index+1)*self.batch_size]
        batch_labels = self.labels[index*self.batch_size:(index+1)*self.batch_size]

        batch_encodings = self.tokenizer.batch_encode_plus(
            batch_texts,
            add_special_tokens=True,
            padding='longest',
            truncation=True,
            max_length=128,
            return_tensors='tf'
        )

        return dict(batch_encodings), batch_labels

    def on_epoch_end(self):
        pass

# Set the hyperparameters
num_epochs = 6
batch_size = 16

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Load and preprocess the data
train_dataset = RedditDataset(x_train, y_train, tokenizer, batch_size=batch_size)
test_dataset = RedditDataset(x_test, y_test, tokenizer, batch_size=batch_size)

# Load the pre-trained BERT model for sentiment analysis
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
model.dropout = tf.keras.layers.Dropout(0.6)  # Add a dropout layer with dropout rate of 0.5

# Define the optimizer
optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=1e-5, decay=0.01)
loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# Compile the model
model.compile(optimizer=optimizer, loss=loss,
              metrics=['accuracy'])

# Train the model
model.fit(train_dataset, epochs=num_epochs, validation_data=test_dataset)

# Evaluate the model
loss, accuracy = model.evaluate(test_dataset)
print(f'Test Loss: {loss:.4f}')
print(f'Test Accuracy: {accuracy * 100:.2f}%')


All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/6
250/250 [==============================] - 142s 457ms/step - loss: 0.6753 - accuracy: 0.5805 - val_loss: 0.5741 - val_accuracy: 0.7200
Epoch 2/6
250/250 [==============================] - 112s 449ms/step - loss: 0.5372 - accuracy: 0.7545 - val_loss: 0.5151 - val_accuracy: 0.7500
Epoch 3/6
250/250 [==============================] - 115s 460ms/step - loss: 0.4761 - accuracy: 0.7955 - val_loss: 0.4993 - val_accuracy: 0.7513
Epoch 4/6
250/250 [==============================] - 112s 449ms/step - loss: 0.4388 - accuracy: 0.8192 - val_loss: 0.4941 - val_accuracy: 0.7650
Epoch 5/6
250/250 [==============================] - 112s 449ms/step - loss: 0.4028 - accuracy: 0.8355 - val_loss: 0.4942 - val_accuracy: 0.7625
Epoch 6/6
50/50 [==============================] - 8s 151ms/step - loss: 0.4945 - accuracy: 0.7600
Test Loss: 0.4945
Test Accuracy: 76.00%
